In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import torch
import numpy as np
import datetime
import glob
import os
import pandas as pd

from torch.utils.data import DataLoader

from model.custom_model import CustomAudioCLIP
from utils.audiolist import AudioList

# Parse the folders (match audio path with result path)

In [30]:
def remove_extension(input):

    filename = input.split("/")[-1].split(".")[0]

    if len(filename) > 2:
        filename = ".".join(filename[0:-1])
    else:
        filename = input.split("/")[-1].split(".")[0]

    return filename

def parseFolders(apath, rpath):

    audio_files = [f for f in glob.glob(apath + "/**/*", recursive = True) if os.path.isfile(f)]
    audio_no_extension = []
    for audio_file in audio_files:
        audio_file_no_extension = remove_extension(audio_file)
        audio_no_extension.append(audio_file_no_extension)


    result_files = [f for f in glob.glob(rpath + "/**/*", recursive = True) if os.path.isfile(f)]

    flist = []
    for result in result_files:
        result_no_extension = remove_extension(result)
        is_in = result_no_extension in audio_no_extension

        if is_in:
            audio_idx = audio_no_extension.index(result_no_extension)
            pair = {'audio': audio_files[audio_idx], 'result': result}
            flist.append(pair)
        else:
            continue

    print('Found {} audio files with valid result file.'.format(len(flist)))

    return flist


In [10]:
df_all = {"Selection": [], "View": [], "Channel": [], "Begin Time (s)": [], "End Time (s)": [], "Low Freq (Hz)": [], "High Freq (Hz)": [], "type": [], "comment": []}
df_all = pd.DataFrame(df_all)

In [ ]:
# NEED TO RUN THE MODEL FIRST ON THE DATA
# THEN CROSS CHECK THE DETECTIONS

# Run the model on the audio files 

In [59]:
import numpy as np
import torch
import yaml
import itertools
import glob
import pandas as pd

from torch.quantization import quantize_dynamic
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import LabelEncoder

from utils.audio_processing import openAudioFile, splitSignal

class AudioList():

    def __init__(self, length_segments = 3, overlap = 0, sample_rate=44100):
        self.sample_rate = sample_rate
        self.length_segments = length_segments
        self.overlap = overlap
        
    def read_audio(self, audio_path):
        """Read the audio, change the sample rate and randomly pick one channel"""
        sig, _ = openAudioFile(audio_path, sample_rate=self.sample_rate)
        return sig

    def split_segment(self, array):
        splitted_array = splitSignal(array, rate=self.sample_rate, seconds=self.length_segments, overlap=self.overlap, minlen=3)
        return splitted_array

    def get_processed_list(self, audio_path):

        list_segments = []
        track = self.read_audio(audio_path)        
        list_divided = self.split_segment(track)
        print(list_divided)
        list_segments.append(list_divided)
        return list_segments

class AudioLoader(Dataset):
    def __init__(self, list_data, sr=44100, transform=None):
        self.data = list_data
        self.transform = transform
        self.sr=sr

    def __len__(self):
        return len(self.data)

    def process_data(self, data):

        array = data
        array = array.reshape(1, -1)
        array = torch.tensor(array)
        return array

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        tensor = self.process_data(self.data[idx])
        return tensor 

def getPredLoader(list_arrays):
    list_preds = AudioLoader(list_arrays)
    predLoader = DataLoader(list_preds, batch_size=1, num_workers=4, pin_memory=False)
    return predLoader

def predict(testLoader, model):

    proba_list = []

    for array in testLoader:
        tensor = array
        output = model(tensor)
        output = np.exp(output.detach().numpy())
        proba_list.append(output[0][1])

    return (np.array(proba_list))

def initModel(mpath):
    m = CustomAudioCLIP(num_target_classes=2).load_from_checkpoint(mpath, num_target_classes=2)
    return m

In [54]:
label_path = "/Data/test_dataset/labels"
audio_test_path = "/Data/test_dataset/audio"

In [60]:
l_audio = AudioList().get_processed_list(flist[0]['audio'])

/opt/conda/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


[array([ 0.        ,  0.        ,  0.        , ..., -0.00709534,
       -0.00724792, -0.00735474], dtype=float32), array([-0.00726318, -0.00717163, -0.00709534, ...,  0.00415039,
        0.00401306,  0.0037384 ], dtype=float32), array([0.00369263, 0.00379944, 0.00379944, ..., 0.00086975, 0.00088501,
       0.0009613 ], dtype=float32), array([0.0009613 , 0.00094604, 0.00102234, ..., 0.00183105, 0.00186157,
       0.00184631], dtype=float32), array([ 0.00169373,  0.00163269,  0.0018158 , ..., -0.00454712,
       -0.00465393, -0.00469971], dtype=float32), array([-0.00471497, -0.00473022, -0.00471497, ...,  0.01095581,
        0.01098633,  0.01109314], dtype=float32), array([0.01107788, 0.01098633, 0.01107788, ..., 0.0040741 , 0.00402832,
       0.00404358], dtype=float32), array([0.00413513, 0.0042572 , 0.00428772, ..., 0.00068665, 0.00074768,
       0.00079346], dtype=float32), array([0.00073242, 0.00059509, 0.00057983, ..., 0.00187683, 0.00198364,
       0.00215149], dtype=float32), arr

In [48]:
flist = parseFolders(audio_test_path, label_path)

for i in range(len(flist)):
    filename = flist[i]['audio']
    AudioList().get_processed_list(filename)

Found 67 audio files with valid result file.


/opt/conda/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


IndexError: list index out of range

In [37]:
flist[0]['audio']

'/Data/test_dataset/audio/YELLPAYP_20130203_171036.MP3005.mp3'

In [45]:
pd.read_csv(flist[5]['result'], sep='\t').filter(items=["Begin Time (s)", "End Time (s)"])

,Begin Time (s),End Time (s)
0,98.991993,116.325299
1,98.991993,116.325299
2,148.106717,216.665269
3,148.106717,216.665269
4,294.082802,309.237341
5,294.082802,309.237341
6,631.682612,649.839008
7,631.682612,649.839008
8,661.277048,682.241633
9,661.277048,682.241633


In [6]:

m = initModel("/app/assets/ckpt-epoch=21-val_loss=0.12-lr=0.005.ckpt")



# OTHER SCRIPTS

In [81]:
import numpy as np
import torch
import yaml
import itertools
import glob
import pandas as pd

from torch.quantization import quantize_dynamic
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import LabelEncoder

from utils import openAudioFile, splitSignal

class AudioListNoLabels():

    def __init__(self, length_segments = 3, sample_rate=44100):
        self.sample_rate = sample_rate
        self.length_segments = length_segments
        
    def read_audio(self, audio_path):
        """Read the audio, change the sample rate and randomly pick one channel"""
        sig, _ = openAudioFile(audio_path, sample_rate=self.sample_rate)
        return sig

    def split_segment(self, array):
        splitted_array = splitSignal(array, rate=self.sample_rate, seconds=self.length_segments, overlap=0, minlen=3)
        return splitted_array

    def get_processed_list(self, audio_path):

        list_segments = []

        for item in audio_path:
            track = self.read_audio(item)        
            label = item.split("/")[-2]
            list_divided = self.split_segment(track)
            list_segments.append(list_divided)
        return list_segments

class AudioLoader(Dataset):
    def __init__(self, list_data, label_encoder, sr=44100, transform=None):
        self.data = list_data
        self.label_encoder = label_encoder
        self.transform = transform
        self.sr=sr

    def __len__(self):
        return len(self.data)

    def process_data(self, data):

        array, label = data
        array = array.reshape(1, -1)
        array = torch.tensor(array)

        label_encoded = self.label_encoder.one_hot_sample(label)
        label_class = torch.argmax(label_encoded)

        return (array, label_class)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        tensor, label = self.process_data(self.data[idx])
        return tensor, label

    def get_labels(self):
        list_labels = []
        for x,y in self.data:
            list_labels.append(y)
        return list_labels

def initModel(model_path):
    m = torch.load(model_path).eval()
    m_q = quantize_dynamic(m, qconfig_spec={torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
    return m_q
    
def getPredLoader(list_arrays, l):
    list_preds = AudioLoader(list_arrays, label_encoder)
    predLoader = DataLoader(list_preds, batch_size=1, num_workers=4, pin_memory=False)
    return predLoader

def predict(testLoader, model):

    proba_list = []
    label_list = []

    for array, label in testLoader:
        tensor = torch.tensor(array)
        output = model(tensor)
        output = np.exp(output.detach().numpy())
        proba_list.append(output[0][1])
        label_list.append(label[0])

    return (np.array(proba_list), np.array(label_list))

In [47]:
with open("/app/prediction_scripts/config.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [48]:

###################
# Get the dataset #
###################

allFiles = [f for f in glob.glob(config["INPUT_PATH"] + "/**/*", recursive=True) if os.path.isfile(f)]
allFiles = [f for f in allFiles if f.endswith( (".WAV", ".wav", ".mp3") )]

# Instantiate the audio iterator class - cut the audio into segments
audio_list= AudioList(length_segments=config["SIG_LENGTH"], sample_rate=config["SAMPLE_RATE"])

list_test = audio_list.get_processed_list(allFiles)
list_test = list(itertools.chain.from_iterable(list_test))

###########################
# Create the labelEncoder #
###########################
label_encoder = EncodeLabels(path_to_folders=config["INPUT_PATH"])

# Save name of the folder and associated label in a json file
l = label_encoder.__getLabels__()
t = label_encoder.class_encode.transform(l)

folder_labels = []
for i, j in zip(l,t):
    item = {"Folder": i, "Label": int(j)}
    folder_labels.append(item)

In [79]:
audioloader = AudioLoader(list_test, label_encoder)
predLoader = DataLoader(audioloader, batch_size=1, num_workers=4, pin_memory=False)
model = initModel(config["MODEL"])
proba_list, labels = predict(predLoader, model)

/tmp/ipykernel_9453/222997309.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(array)


In [92]:
precision, recall, thresholds = precision_recall_curve(labels, proba_list)
thresholds = np.append(thresholds, 1)

In [94]:
d = {'thresholds': thresholds, 'precision': precision, 'recall': recall}
df = pd.DataFrame(data=d)

: 

In [14]:
import pandas as pd
import glob
import os

path = '/Data/audioCLIP/results' # use your path
all_files = glob.glob(path + "/*")
print(all_files)
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df_conf = df[df["confidence"] > 0.99]
    print(len(df_conf))
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

['/Data/audioCLIP/results/YELLMACS_20120131_105013.csv', '/Data/audioCLIP/results/YELLPAYP_20130124_083717.csv', '/Data/audioCLIP/results/YELLMJ2B_20140327_154332.csv', '/Data/audioCLIP/results/YELLSYL3_20130206_223122.csv', '/Data/audioCLIP/results/YELLMM8K_20080214_021244.csv', '/Data/audioCLIP/results/YELLSYL3_20130207_075037.csv', '/Data/audioCLIP/results/YELLFOPP_20141229_110225.csv', '/Data/audioCLIP/results/YELLMJ23_20111229_154155.csv']
275
808
37
0
0
13
568
578


In [17]:
! ffmpeg -i /Data/audioCLIP/files_to_split/YELLCRPA_20110112_060550.MP3 -c copy -map 0 -segment_time 00:30:00 -f segment /Data/audioCLIP/output%03d.mp3

ffmpeg version 4.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/opt/conda --cc=/opt/conda/conda-bld/ffmpeg_1597178665428/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-pic --enable-pthreads --enable-shared --disable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
[mp3 @ 0x55be63d6ccc0] Estimating duration from bitrate, this may be inaccurate
Input #0, mp3, from '/Data/audioCLIP/files_to_split/YELLCRPA_20110112_060550.MP3':
  Durat